In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.4 MB/s eta 0:00:00


In [3]:
def shuffle_dataset(data_X,data_Y):
    length = len(data_X)
    rng = np.random.default_rng(12345)
    index = np.arange(length)
    print(index)
    rng.shuffle(index)
    print(index)
    return data_X[index],data_Y[index]

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

title_0_embed = np.load('/content/drive/MyDrive/neckpain_vector/title0_embed.npy')
title_1_embed = np.load('/content/drive/MyDrive/neckpain_vector/title1_embed.npy')
abstruct_0_embed = np.load('/content/drive/MyDrive/neckpain_vector/abstract0_embed.npy')
abstruct_1_embed = np.load('/content/drive/MyDrive/neckpain_vector/abstract1_embed.npy')
X_0 = pd.read_excel('/content/drive/MyDrive/neckpain_data/X_0.xlsx')
X_1 = pd.read_excel('/content/drive/MyDrive/neckpain_data/X_1.xlsx')
Y_0 = pd.read_excel('/content/drive/MyDrive/neckpain_data/Y_0.xlsx').values
Y_1 = pd.read_excel('/content/drive/MyDrive/neckpain_data/Y_1.xlsx').values

title_0 = pd.DataFrame(title_0_embed)
title_1 = pd.DataFrame(title_1_embed)
abstruct_0 = pd.DataFrame(abstruct_0_embed)
abstruct_1 = pd.DataFrame(abstruct_1_embed)

feature = pd.concat([X_0,title_0], axis=1)
feature_0 = pd.concat([feature,abstruct_0], axis=1).values
feature = pd.concat([X_1,title_1], axis=1)
feature_1 = pd.concat([feature,abstruct_1], axis=1).values

rs = np.random.RandomState(42)
L = list(rs.randint(0, len(X_0), int(7/3*len(X_1))))
feature_0 = feature_0[L]
Y_0 = Y_0[L]

train_X_0, test_X_0, train_Y_0, test_Y_0 = train_test_split(feature_0, Y_0, train_size=0.80, random_state=42)
train_X_1, test_X_1, train_Y_1, test_Y_1 = train_test_split(feature_1, Y_1, train_size=0.80, random_state=42)

train_Y_0 = train_Y_0.reshape(len(train_Y_0),1)
train_Y_1 = train_Y_1.reshape(len(train_Y_1),1)
test_Y_0 = test_Y_0.reshape(len(test_Y_0),1)
test_Y_1 = test_Y_1.reshape(len(test_Y_1),1)

train_X = np.vstack((train_X_0,train_X_1))
train_Y = np.vstack((train_Y_0,train_Y_1))
test_X = np.vstack((test_X_0,test_X_1))
test_Y = np.vstack((test_Y_0,test_Y_1))

train_X_input,train_Y_input = shuffle_dataset(train_X,train_Y)
test_X_input,test_Y_input = shuffle_dataset(test_X,test_Y)

[   0    1    2 ... 2677 2678 2679]
[ 347 1822  200 ... 1246  397  413]
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228

In [10]:
!pip install thop
!pip install torch

In [16]:
from sklearn.linear_model import LassoCV, Lasso
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, roc_auc_score, confusion_matrix
import warnings
import time
import thop
from torch.profiler import profile, record_function, ProfilerActivity
import torch

warnings.filterwarnings(action = 'ignore')

Lambdas = np.logspace(-5,2,200)
lasso_cv = LassoCV(alphas=Lambdas, cv=5, max_iter=5000)
lasso_cv.fit(train_X_input,train_Y_input)
lasso = Lasso(alpha=lasso_cv.alpha_, max_iter=10000)

start_time = time.time()
lasso.fit(train_X_input,train_Y_input)
end_time = time.time()

# 计算模型的FLOPs和参数数量
flops = 2 * train_X_input.shape[1] * train_X_input.shape[0]
print(f"FLOPs: {flops}")


# 计算每秒钟的训练样本数（即MLOPS）
num_samples = len(train_X_input)
training_time = end_time - start_time
samples_per_second = num_samples / training_time

y_pred_lasso = lasso.predict(test_X_input)
print('lasso AUC = ',metrics.roc_auc_score(test_Y_input,y_pred_lasso))

for i in range(y_pred_lasso.size):
    if(y_pred_lasso[i] > 0.5):
        y_pred_lasso[i] = 1
    else:
        y_pred_lasso[i] = 0
print(metrics.classification_report(y_true=test_Y_input,y_pred= y_pred_lasso, digits=4))
tn, fp, fn, tp = confusion_matrix(test_Y_input, y_pred_lasso).ravel()
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)
print('Specificity = ', specificity)
print('Sensitivity = ', sensitivity)

# 输出模型信息
print("MLOPS: {:.2f}".format(samples_per_second))

FLOPs: 8318720
lasso AUC =  0.8564321250888415
              precision    recall  f1-score   support

           0     0.8189    0.9254    0.8689       469
           1     0.7500    0.5224    0.6158       201

    accuracy                         0.8045       670
   macro avg     0.7844    0.7239    0.7424       670
weighted avg     0.7982    0.8045    0.7930       670

Specificity =  0.9253731343283582
Sensitivity =  0.5223880597014925
MLOPS: 54.75


In [20]:
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

# from sklearn.metrics import f1_score,recall_score,precision_score,accuracy_score
from sklearn.metrics import classification_report
# from sklearn.metrics import roc_curve, auc,accuracy_score,precision_recall_curve

B = 800
n_classes = 2
RF = RandomForestClassifier(n_estimators=B,class_weight={0:0.3,1:0.7},
                            max_depth=50,bootstrap = True,oob_score=True, max_features='sqrt',random_state=0)
start_time = time.time()
RF.fit(train_X_input,train_Y_input)
end_time = time.time()
# 计算每秒钟的训练样本数（即MLOPS）
num_samples = len(train_X_input)
training_time = end_time - start_time
samples_per_second = num_samples / training_time

prob = RF.predict_proba(test_X_input)[:,1]
print('RandomForest AUC = ',metrics.roc_auc_score(test_Y_input,prob))
# 计算单个决策树的FLOPs
n_nodes = 2 ** RF.max_depth - 1
n_leaves = 2 ** (RF.max_depth - 1)
n_features = test_X_input.shape[1]
flops_per_tree = (n_features * n_leaves * np.log2(n_leaves) +
                  n_leaves * n_classes * (n_classes - 1) +
                  n_nodes * n_features * np.log2(n_leaves))

# 计算随机森林的FLOPs
flops = flops_per_tree * RF.n_estimators
print(f"FLOPs: {flops}")
print("MLOPS: {:.2f}".format(samples_per_second))

predict = []
for i in range(len(test_Y_input)):
  if(prob[i]>0.3):
    predict.append(1)
  else:
    predict.append(0)
tn, fp, fn, tp = confusion_matrix(test_Y_input, predict).ravel()
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)
print('Specificity = ', specificity)
print('Sensitivity = ', sensitivity)
print(metrics.classification_report(y_true=test_Y_input,y_pred= predict, digits=4))

RandomForest AUC =  0.8528519449660015
FLOPs: 1.0274782405860685e+23
MLOPS: 30.23
Specificity =  0.7654584221748401
Sensitivity =  0.7910447761194029
              precision    recall  f1-score   support

           0     0.8953    0.7655    0.8253       469
           1     0.5911    0.7910    0.6766       201

    accuracy                         0.7731       670
   macro avg     0.7432    0.7783    0.7509       670
weighted avg     0.8040    0.7731    0.7807       670

